In [133]:
import numpy as np

def square_root(A):
    n = A.shape[0]
    U = np.zeros((n, n), dtype=np.complex128)
    
    for i in range(n):
        for j in range(i, n):
            s = sum(U[k, i].conj() * U[k, j] for k in range(i))
            
            if i == j:
                U[i, i] = np.sqrt(A[i, i] - s)
            else:
                U[i, j] = (A[i, j] - s) / U[i, i]
    return U

def solve_upper_triangular(U, b):
    n = len(b)
    x = np.zeros(n, dtype=np.complex128)
    for i in reversed(range(n)):
        x[i] = (b[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x

def solve_lower_triangular(L, b):
    n = len(b)
    x = np.zeros(n, dtype=np.complex128)
    for i in range(n):
        x[i] = (b[i] - np.dot(L[i, :i], x[:i])) / L[i, i]
    return x

A = np.array([
    [0.83, 2.18, -1.73],
    [2.18, -1.41, 1.03],
    [-1.73, 1.03, 2.27]
], dtype=np.float64)

y = np.array([0.83, -1.12, 0.47], dtype=np.float64)

assert np.allclose(A, A.T), "Матрица не симметрична"

U = square_root(A)

z = solve_lower_triangular(U.T.conj(), y)
x = solve_upper_triangular(U, z)

x_real = np.real_if_close(x)

x_exact =  np.linalg.solve(A, y)

residual = np.linalg.norm(A @ x_real - y)

eigenvalues = np.linalg.eigvals(A)
is_positive_definite = np.all(eigenvalues > 0)

print(f"Все собственные значения положительны: {'Да, значит метод применим' if is_positive_definite else 'Нет, значит метод не применим'}")
print(f"Собственные значения: {eigenvalues.round(3)}")
print("-" * 40)

print("\nРазложение U:")
print(np.round(U, 4))
print("-" * 40)

print("\nПриближенное решение:")
print(f"x = {np.round(x_real, 6)}")
print(f"Норма невязки: {residual:.6f}")
print("\nТочное решение (Wolfram):")
print(f"x = {x_exact}")
print("-" * 40)

Все собственные значения положительны: Нет, значит метод не применим
Собственные значения: [-3.324  3.461  1.554]
----------------------------------------

Разложение U:
[[ 0.911 +0.j      2.3929+0.j     -1.8989+0.j    ]
 [ 0.    +0.j      0.    +2.6713j  0.    -2.0866j]
 [ 0.    +0.j      0.    +0.j      0.    +2.3853j]]
----------------------------------------

Приближенное решение:
x = [ 2.212476 -0.514307 -0.066378]
Норма невязки: 8.266829

Точное решение (Wolfram):
x = [-0.21874575  0.36470614 -0.12514427]
----------------------------------------


In [135]:
import numpy as np

def simple_iteration_with_steps(B, c, epsilon=0.001, max_iter=1000):
    n = len(c)
    x_prev = np.zeros(n)
    history = []
    
    B_norm = np.max(np.sum(np.abs(B), axis=0))
    spectral_radius = np.max(np.abs(np.linalg.eigvals(B)))
    
    print("Теоретические проверки:")
    print(f"1. Достаточное условие: ||B||_1 = {B_norm:.2f} {'< 1' if B_norm < 1 else '≥ 1'}")
    print(f"2. Необходимое условие: ρ(B) = {spectral_radius:.2f} {'< 1' if spectral_radius < 1 else '≥ 1'}")
    
    if B_norm >= 1 and spectral_radius >= 1:
        raise ValueError("Метод гарантированно расходится: оба условия нарушены")
    elif spectral_radius >= 1:
        print("ρ(B) ≥ 1. Метод может расходиться!")
    elif B_norm >= 1:
        print("||B||₁ ≥ 1. Сходимость не гарантируется, но возможна")
    else:
        print("Оба условия выполнены. Метод гарантированно сходится")
    
    print("-" * 40)
    
    for iter in range(max_iter):
        x_current = B @ x_prev + c
        diff = x_current - x_prev
        norm = np.max(np.abs(diff))
        
        history.append({
            "iteration": iter + 1,
            "x": x_current.copy(),
            "norm": norm,
            "spectral_radius": spectral_radius
        })
        
        print(f"Итерация {iter + 1}:")
        print(f"x = {np.round(x_current, 4)}")
        print(f"Δx: {norm:.4f}")
        print("-" * 40)
        
        if norm < epsilon:
            break
            
        x_prev = x_current.copy()
    else:
        print(f"Точность не достигнута за {max_iter} итераций")
    
    return history

# ----------------------------------------------------------
B = np.array([
    [0.00, 0.22, -0.11, 0.31],
    [0.38, 0.00, -0.12, 0.22],
    [0.11, 0.23, 0.00, -0.51],
    [0.17, -0.21, 0.31, 0.00]
], dtype=float)

c = np.array([2.7, -1.5, 1.2, -0.17], dtype=float)

history = simple_iteration_with_steps(B, c, epsilon=0.001)

final_solution = history[-1]["x"]
print("\nФинальное решение (x₁-x₄):")
for i, x in enumerate(final_solution.round(3), 1):
    print(f"x{i} = {x:.3f}")

Теоретические проверки:
1. Достаточное условие: ||B||_1 = 1.04 ≥ 1
2. Необходимое условие: ρ(B) = 0.46 < 1
||B||₁ ≥ 1. Сходимость не гарантируется, но возможна
----------------------------------------
Итерация 1:
x = [ 2.7  -1.5   1.2  -0.17]
Δx: 2.7000
----------------------------------------
Итерация 2:
x = [ 2.1853 -0.6554  1.2387  0.976 ]
Δx: 1.1460
----------------------------------------
Итерация 3:
x = [ 2.7221 -0.6035  0.7919  0.7231]
Δx: 0.5368
----------------------------------------
Итерация 4:
x = [ 2.7043 -0.4015  0.9918  0.665 ]
Δx: 0.2020
----------------------------------------
Итерация 5:
x = [ 2.7087 -0.4451  1.066   0.6815]
Δx: 0.0742
----------------------------------------
Итерация 6:
x = [ 2.6961 -0.4487  1.048   0.7144]
Δx: 0.0329
----------------------------------------
Итерация 7:
x = [ 2.7075 -0.4441  1.029   0.7074]
Δx: 0.0190
----------------------------------------
Итерация 8:
x = [ 2.7084 -0.439   1.0349  0.7025]
Δx: 0.0059
--------------------------------

In [137]:
import numpy as np

def seidel_method_v2(A, b, epsilon=0.001, max_iter=100):
    n = len(b)
    x = np.zeros(n)
    
    D = np.diag(np.diag(A))
    U = np.triu(A, 1)  
    L = np.tril(A, -1)  
    
    try:
        F = -np.linalg.inv(D + U) @ L
    except np.linalg.LinAlgError:
        raise ValueError("Матрица (D+U) вырождена")
    
    F_norm = np.max(np.sum(np.abs(F), axis=0))  # Норма ||F||₁
    spectral_radius = np.max(np.abs(np.linalg.eigvals(F)))
    
    print("Теоретические проверки:")
    print(f"1. Достаточное условие (||F||₁ < 1): {F_norm:.4f} {'✓' if F_norm < 1 else '✗'}")
    print(f"2. Необходимое условие (ρ(F) < 1): {spectral_radius:.4f} {'✓' if spectral_radius < 1 else '✗'}")
    
    if F_norm >= 1 and spectral_radius >= 1:
        print("Оба условия нарушены")
    elif F_norm >= 1:
        print("Сходимость не гарантируется (||F||₁ ≥ 1)")
    elif spectral_radius >= 1:
        print("Сходимость не гарантируется (ρ(F) ≥ 1)")
    else:
        print("Оба условия выполнены")
    print("=" * 50)
    
    for k in range(max_iter):
        x_old = x.copy()
        
        for i in reversed(range(n)):
            sum_L = np.dot(A[i,:i], x_old[:i])
            sum_U = np.dot(A[i,i+1:], x[i+1:])
            x[i] = (b[i] - sum_L - sum_U) / A[i,i]
        
        delta = np.max(np.abs(x - x_old))
        print(f"Итерация {k+1}:")
        print(f"x = {np.round(x, 4)}")
        print(f"Δx: {delta:.4f}")
        print("-" * 50)
        
        if delta < epsilon:
            break
    
    return np.round(x, 3)

A = np.array([
    [5.6,  2.7, -1.7],
    [3.4, -3.6, -6.7],
    [0.8,  1.3,  3.7]
], dtype=float)

b = np.array([1.9, -2.4, 1.2], dtype=float)

solution = seidel_method_v2(A, b, epsilon=0.001)

print("\nФинальное решение:")
for i, val in enumerate(solution, 1):
    print(f"x{i} = {val:.3f}")

Теоретические проверки:
1. Достаточное условие (||F||₁ < 1): 2.2781 ✗
2. Необходимое условие (ρ(F) < 1): 0.3174 ✓
Сходимость не гарантируется (||F||₁ ≥ 1)
Итерация 1:
x = [0.4073 0.0631 0.3243]
Δx: 0.4073
--------------------------------------------------
Итерация 2:
x = [0.0895 0.6529 0.2141]
Δx: 0.5899
--------------------------------------------------
Итерация 3:
x = [0.0679 0.6105 0.0756]
Δx: 0.1385
--------------------------------------------------
Итерация 4:
x = [0.1012 0.5537 0.0951]
Δx: 0.0568
--------------------------------------------------
Итерация 5:
x = [0.1013 0.5614 0.1079]
Δx: 0.0128
--------------------------------------------------
Итерация 6:
x = [0.098  0.5667 0.1051]
Δx: 0.0053
--------------------------------------------------
Итерация 7:
x = [0.0982 0.5656 0.104 ]
Δx: 0.0011
--------------------------------------------------
Итерация 8:
x = [0.0985 0.5651 0.1044]
Δx: 0.0005
--------------------------------------------------

Финальное решение:
x1 = 0.098
x2 = 0

In [141]:
import numpy as np

def gradient_descent_with_checks(A, b, epsilon=1e-4, max_iter=10000):
    n = len(b)
    x = np.zeros(n, dtype=np.float64)
    
    A_new = A.T @ A
    b_new = A.T @ b
    
    eigenvalues = np.linalg.eigvals(A_new)
    M = np.max(eigenvalues)
    m = np.min(eigenvalues)
    cond_number = M / m
    
    print("Теоретические параметры:")
    print(f"Спектральный радиус: {(M - m)/(M + m):.4f}")
    print(f"Число обусловленности: {cond_number:.2f}")
    print("=" * 50)
    
    for k in range(max_iter):
        r = b_new - A_new @ x
        Ar = A_new @ r
        dzeta = np.dot(r, r) / np.dot(r, Ar)
        x_new = x + dzeta * r
        
        delta = np.linalg.norm(x_new - x)
        residual = np.linalg.norm(r)
        
        print(f"Итерация {k+1}:")
        print(f"x = {np.round(x, 4)}")
        print(f"Δx: {delta:.4e}")
        print(f"Норма невязки: {residual:.4e}")
        print("-" * 50)
        
        x = x_new.copy()
        if residual < epsilon:
            break
    
    x_exact = np.linalg.solve(A, b)
    error = np.linalg.norm(x - x_exact)
    
    print("\nФинальные результаты:")
    print(f"Выполнено итераций: {k+1}")
    print(f"Норма ошибки: {error:.2e}")
    print("Точное решение:", np.round(x_exact, 3))
    return np.round(x, 3)

A = np.array([
    [0.73, 1.24, -0.38, -1.43],
    [1.07, -0.77, 1.25, 0.66], 
    [1.56, 0.66, 1.44, -0.87],
    [0.75, 1.22, -0.83, 0.37]
], dtype=np.float64)

b = np.array([0.58, -0.66, 1.24, 0.92], dtype=np.float64)

solution = gradient_descent_with_checks(A, b)

print("\nПриближенное решение:")
for i, val in enumerate(solution, 1):
    print(f"x{i} = {val:.3f}")

Теоретические параметры:
Спектральный радиус: 0.9782
Число обусловленности: 90.84
Итерация 1:
x = [0. 0. 0. 0.]
Δx: 5.6190e-01
Норма невязки: 4.4198e+00
--------------------------------------------------
Итерация 2:
x = [ 0.2977  0.4028 -0.003  -0.2547]
Δx: 1.2006e-01
Норма невязки: 5.2236e-01
--------------------------------------------------
Итерация 3:
x = [ 0.2474  0.4752 -0.0617 -0.1982]
Δx: 1.0248e-01
Норма невязки: 3.6135e-01
--------------------------------------------------
Итерация 4:
x = [ 0.2778  0.5238  0.0124 -0.1566]
Δx: 7.9392e-02
Норма невязки: 3.5692e-01
--------------------------------------------------
Итерация 5:
x = [ 0.217   0.5514 -0.0033 -0.1164]
Δx: 8.1573e-02
Норма невязки: 3.0240e-01
--------------------------------------------------
Итерация 6:
x = [ 0.2316  0.6075  0.0537 -0.1105]
Δx: 7.0728e-02
Норма невязки: 3.2442e-01
--------------------------------------------------
Итерация 7:
x = [ 0.1722  0.6185  0.0542 -0.0737]
Δx: 7.6210e-02
Норма невязки: 2.8494